In [ ]:
# Parse ucm digits file to json
# parse the list in the rando, compare with the digits file, and if they habe a model listed calc the filesize
# parse the locations list in the rando, calculate file size and output to json
# write a yaml writer/reader that can generate replacement codes, using english or code name
# do testing related to file sizes, figure out the bounds
ucm = open(os.path.join("research","ucm.paste")).read().split("\n")
us = []
for line in ucm:
    l = line.split("-")
    num = l[0].strip()
    descr = ''.join(l[1:]).strip()
    model = descr.split("\t")[-1] if "\t" in descr else ''
    descr = descr.split("\t")[0] if "\t" in descr else descr
    if model:
        model = model.split('("')[1].split('")')[0]
    us.append((num,descr,model))
with open("ucm.json","w") as f:
    f.write("ucm,descr,model\n")
    for l in us:
        f.write("{},{},{}\n".format(l[0], l[1], l[2]))
us

In [ ]:
# Need a list of UCMs that are bosses or enemies, so I can filter properly (and list if they are boss or enemy)
x = []
f = open("bosstable.csv").read().split("\n")
commands = f[0].split(",")
for line in f[1:]:
    l = line.split(",")
    r = {}
    for i in range(len(commands)):
        r[commands[i]] = l[i]
    x.append(r)
print(x)

In [ ]:
records = []
for boss in x:
    code = boss["Obj Code"]
    aiparam = ''.join(code[:2]) if len(code) == 6 else ''
    code = code if not aiparam else ''.join(code[2:])
    otherboss = d[code]
    record = {
        "ucm": code,
        "name": boss["Boss Name"],
        "model": otherboss["model"],
        "animation": "", 
        "size": "",
        "aiparam": aiparam,
        "type": "boss",
    }
    records.append(record)

In [ ]:
enemies = {}
for location in ens:
    for enemy in location["enemies"]:
        code = enemy["enemy"]["value"]
        aiparam = ''.join(code[:2]) if len(code) == 6 else ''
        code = code if not aiparam else ''.join(code[2:])
        otherboss = d[code]
        record = {
            "ucm": code,
            "name": enemy["enemy"]["name"],
            "model": otherboss["model"],
            "animation": "",
            "size": "",
            "aiparam": aiparam,
            "type": "enemy"
        }
        enemies[code] = record
for record in enemies.values():
    records.append(record)
records

In [ ]:
with open(os.path.join("research", "enemy_records.csv"), "w") as f:
    json.dump(records, f)

In [ ]:
d = {}
for i in us:
    d[i[0]] = {"name": i[1], "model": i[2]}

In [ ]:
# read file into json
# for each line add up the filesize of the model and anim file
  # if extra spawns recurse for that size
  # if the ucms or spawn ucms are not 4 length, pad them
  # if no anim filename, use model name
  # if no anim fileexists don't try to read it
  # optionally include the .fm file based on the model name
# write this out to json/csv
OBJDIR = os.path.join(os.environ["USE_KH2_GITPATH"], "KH2", "obj")
lines = open(os.path.join("research","enemy_records_filled.csv")).read().split("\n")
headers = lines[0].split(",")
records = {}
for line in lines[1:]:
    if len(line) == 0:
        continue
    record = {}
    for i in range(len(headers)):
        record[headers[i]] = line.split(",")[i]
    record["ucm"] = record["ucm"].zfill(4)
    record["spawns"] = [i.zfill(4) for i in record["spawns"].split(';') if len(i) > 0]
    name = record["name"].strip()
    record["name"] = name
    records[name] = record
def getsize(name):
    size = 0
    record = records[name]
    size = os.path.getsize(os.path.join(OBJDIR,record["model"]+".mdlx"))
    fmfn = os.path.join(OBJDIR,record["model"]+".fm")
    if os.path.exists(fmfn):
        size += os.path.getsize(fmfn)
    anfn = record["animation"]
    if len(anfn) == 0:
        anfn = record["model"] + ".mset"
    anfn = os.path.join(OBJDIR, anfn)
    if os.path.exists(anfn):
        size += os.path.getsize(anfn)
    for spawn in record["spawns"]:
        for r in records:
            if records[r]["ucm"] == spawn:
                size += getsize(records[r]["name"])
    return size
for r in records:
    records[r]["size"] = getsize(r)
records
json.dump(records, open(os.path.join("research", "enemy_records_filled.json"), "w"), sort_keys=True, indent=4, separators=(',', ': '))

In [ ]:
for r in sorted(records.values(), key=lambda k: k["size"]):
    print("{} {}".format(r["size"], r["name"]))

In [ ]:
def lookupEnemySize(enemy):
    if len(enemy) == 4:
        aiparam = ''
    if len(enemy) == 6:
        aiparam = enemy[1]
        enemy = enemy[2:]
    for e in records.values():
        if e["ucm"].lower().zfill(4) == enemy.lower():
            print("hi")
            if e["aiparam"] == '0' or e["aiparam"] == '':
                if aiparam == '0' or aiparam == '':
                    return e["size"]
            if e["aiparam"] == aiparam:
                return e["size"]
    raise Exception("enemy not found {}".format(enemy))
lookupEnemySize('035E')